In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import roc_auc_score

In [2]:
train_features = pd.read_csv('training_set_features.csv')
train_labels = pd.read_csv('training_set_labels.csv')
test_features = pd.read_csv('test_set_features.csv')
submission_format = pd.read_csv('submission_format.csv')

In [61]:
train_features

,respondent_id,xyz_concern,xyz_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,...,income_poverty,marital_status,rent_or_own,employment_status,hhs_geo_region,census_msa,household_adults,household_children,employment_industry,employment_occupation
0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,Below Poverty,Not Married,Own,Not in Labor Force,oxchjgsf,Non-MSA,0.0,0.0,NaN,NaN
1,1,3.0,2.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,...,Below Poverty,Not Married,Rent,Employed,bhuqouqj,"MSA, Not Principle City",0.0,0.0,pxcmvdjn,xgwztkwe
2,2,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,"<= $75,000, Above Poverty",Not Married,Own,Employed,qufhixun,"MSA, Not Principle City",2.0,0.0,rucpziij,xtkaffoo
3,3,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,...,Below Poverty,Not Married,Rent,Not in Labor Force,lrircsnp,"MSA, Principle City",0.0,0.0,NaN,NaN
4,4,2.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,...,"<= $75,000, Above Poverty",Married,Own,Employed,qufhixun,"MSA, Not Principle City",1.0,0.0,wxleyezf,emcorrxb
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26702,26702,2.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,"<= $75,000, Above Poverty",Not Married,Own,Not in Labor Force,qufhixun,Non-MSA,0.0,0.0,NaN,NaN
26703,26703,1.0,2.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,"<= $75,000, Above Poverty",Not Married,Rent,Employed,lzgpxyit,"MSA, Principle City",1.0,0.0,fcxhlnwr,cmhcxjea
26704,26704,2.0,2.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,...,NaN,Not Married,Own,NaN,lzgpxyit,"MSA, Not Principle City",0.0,0.0,NaN,NaN
26705,26705,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,...,"<= $75,000, Above Poverty",Married,Rent,Employed,lrircsnp,Non-MSA,1.0,0.0,fcxhlnwr,haliazsg


In [12]:
train_features["employment_occupation"].info()

<class 'pandas.core.series.Series'>
RangeIndex: 26707 entries, 0 to 26706
Series name: employment_occupation
Non-Null Count  Dtype 
--------------  ----- 
13237 non-null  object
dtypes: object(1)
memory usage: 208.8+ KB


In [3]:
train_labels


,respondent_id,xyz_vaccine,seasonal_vaccine
0,0,0,0
1,1,0,1
2,2,0,0
3,3,0,1
4,4,0,0
...,...,...,...
26702,26702,0,0
26703,26703,0,0
26704,26704,0,1
26705,26705,0,0


In [5]:
train_features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26707 entries, 0 to 26706
Data columns (total 36 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   respondent_id                26707 non-null  int64  
 1   xyz_concern                  26615 non-null  float64
 2   xyz_knowledge                26591 non-null  float64
 3   behavioral_antiviral_meds    26636 non-null  float64
 4   behavioral_avoidance         26499 non-null  float64
 5   behavioral_face_mask         26688 non-null  float64
 6   behavioral_wash_hands        26665 non-null  float64
 7   behavioral_large_gatherings  26620 non-null  float64
 8   behavioral_outside_home      26625 non-null  float64
 9   behavioral_touch_face        26579 non-null  float64
 10  doctor_recc_xyz              24547 non-null  float64
 11  doctor_recc_seasonal         24547 non-null  float64
 12  chronic_med_condition        25736 non-null  float64
 13  child_under_6_mo

In [7]:
train_features.describe()

,respondent_id,xyz_concern,xyz_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,...,health_worker,health_insurance,opinion_xyz_vacc_effective,opinion_xyz_risk,opinion_xyz_sick_from_vacc,opinion_seas_vacc_effective,opinion_seas_risk,opinion_seas_sick_from_vacc,household_adults,household_children
count,26707.000000,26615.000000,26591.000000,26636.000000,26499.000000,26688.000000,26665.000000,26620.00000,26625.000000,26579.000000,...,25903.000000,14433.00000,26316.000000,26319.000000,26312.000000,26245.000000,26193.000000,26170.000000,26458.000000,26458.000000
mean,13353.000000,1.618486,1.262532,0.048844,0.725612,0.068982,0.825614,0.35864,0.337315,0.677264,...,0.111918,0.87972,3.850623,2.342566,2.357670,4.025986,2.719162,2.118112,0.886499,0.534583
std,7709.791156,0.910311,0.618149,0.215545,0.446214,0.253429,0.379448,0.47961,0.472802,0.467531,...,0.315271,0.32530,1.007436,1.285539,1.362766,1.086565,1.385055,1.332950,0.753422,0.928173
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,...,0.000000,0.00000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000
25%,6676.500000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.00000,0.000000,0.000000,...,0.000000,1.00000,3.000000,1.000000,1.000000,4.000000,2.000000,1.000000,0.000000,0.000000
50%,13353.000000,2.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.00000,0.000000,1.000000,...,0.000000,1.00000,4.000000,2.000000,2.000000,4.000000,2.000000,2.000000,1.000000,0.000000
75%,20029.500000,2.000000,2.000000,0.000000,1.000000,0.000000,1.000000,1.00000,1.000000,1.000000,...,0.000000,1.00000,5.000000,4.000000,4.000000,5.000000,4.000000,4.000000,1.000000,1.000000
max,26706.000000,3.000000,2.000000,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,...,1.000000,1.00000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,3.000000,3.000000


In [4]:
train_labels.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26707 entries, 0 to 26706
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype
---  ------            --------------  -----
 0   respondent_id     26707 non-null  int64
 1   xyz_vaccine       26707 non-null  int64
 2   seasonal_vaccine  26707 non-null  int64
dtypes: int64(3)
memory usage: 626.1 KB


In [15]:
train_data = pd.merge(train_features, train_labels, on='respondent_id')

# Drop respondent_id from training and test data
X = train_data.drop(['respondent_id', 'xyz_vaccine', 'seasonal_vaccine','race','rent_or_own','education'], axis=1)
y = train_data[['xyz_vaccine', 'seasonal_vaccine']]
test_data = test_features.drop(['respondent_id'], axis=1)



In [23]:
def fill_missing_values(df):
    for column in df.columns:
        if np.issubdtype(df[column].dtype, np.number):
            df[column].fillna(df[column].median(), inplace=True)
        else:
            df[column].fillna(df[column].mode()[0], inplace=True)
    return df

In [24]:
X_filled = fill_missing_values(X)
test_data_filled = fill_missing_values(test_data)

In [25]:
X_filled = pd.get_dummies(X_filled, drop_first=True)
test_data_filled = pd.get_dummies(test_data_filled, drop_first=True)


In [26]:
X_filled, test_data_filled = X_filled.align(test_data_filled, join='left', axis=1, fill_value=0)

In [29]:
X_train, X_valid, y_train, y_valid = train_test_split(X_filled, y, test_size=0.2, random_state=42)

In [30]:

clf_xyz = SVC(probability=True, random_state=42)
clf_seasonal = SVC(probability=True, random_state=42)

clf_xyz.fit(X_train, y_train['xyz_vaccine'])
clf_seasonal.fit(X_train, y_train['seasonal_vaccine'])

SVC(probability=True, random_state=42)

In [31]:
xyz_pred_proba = clf_xyz.predict_proba(X_valid)[:, 1]
seasonal_pred_proba = clf_seasonal.predict_proba(X_valid)[:, 1]

In [32]:
# Calculate ROC AUC scores
roc_auc_xyz = roc_auc_score(y_valid['xyz_vaccine'], xyz_pred_proba)
roc_auc_seasonal = roc_auc_score(y_valid['seasonal_vaccine'], seasonal_pred_proba)

# Print the average ROC AUC score
print('ROC AUC Score for xyz_vaccine:', roc_auc_xyz)
print('ROC AUC Score for seasonal_vaccine:', roc_auc_seasonal)
print('Average ROC AUC Score:', (roc_auc_xyz + roc_auc_seasonal) / 2)


ROC AUC Score for xyz_vaccine: 0.8136430678466077
ROC AUC Score for seasonal_vaccine: 0.8611900125898957
Average ROC AUC Score: 0.8374165402182516


In [40]:
# Predict probabilities on the test set
test_xyz_pred_proba = clf_xyz.predict_proba(test_data_filled)[:, 1]
test_seasonal_pred_proba = clf_seasonal.predict_proba(test_data_filled)[:, 1]

# Create submission dataframe
submission = pd.DataFrame({
    'respondent_id': test_features['respondent_id'],
    'xyz_vaccine': test_xyz_pred_proba,
    'seasonal_vaccine': test_seasonal_pred_proba
})

# Save to CSV
submission.to_csv('submission.csv', index=False)

In [44]:
submission= pd.read_csv('submission.csv')

In [43]:
submission

,respondent_id,xyz_vaccine,seasonal_vaccine
0,26707,0.109691,0.244942
1,26708,0.119178,0.084212
2,26709,0.261961,0.708618
3,26710,0.647732,0.882924
4,26711,0.126624,0.491404
...,...,...,...
26703,53410,0.276197,0.500000
26704,53411,0.144302,0.241583
26705,53412,0.115706,0.170372
26706,53413,0.087227,0.281518
